In [1]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm, poisson
from scipy.optimize import minimize

# Usando um parâmetro por jogador

In [2]:
def generate_games():
    games = [[chr(i + 65) for i in range(20)] for j in range(20)]
    for i in range(20):
        games[i].pop(i)
        
    return games

def generate_players(temps = 15):
    players = list(np.random.normal(70, 10, 220))
    players = [round(player, 1) if player < 100 else 100 for player in players]
    clubs = {}
    for year in range(temps):
        line_up = {}
        if type(players) != list:
            np.random.shuffle(list(players.flat))

        players = np.reshape(players, (20, 11))
        for club in range(20):
            line_up[chr(65 + club)] = players[club]

        clubs[year] = line_up
        
    return clubs

def find_median(array):
    arraycopy = array.copy()
    arraycopy.sort()
    if len(arraycopy) % 2 == 0:
        return (arraycopy[len(arraycopy) // 2 - 1] + arraycopy[len(arraycopy) // 2])/2
    else:
        return arraycopy[len(arraycopy) // 2]
    
def find_forces(clubs, year = 0):
    atk_forces = []
    def_forces = []
    for club in clubs[year]:
        atk_forces.append(np.sum(clubs[year][club]))
        def_forces.append(np.log(np.prod(clubs[year][club])))
            
    atk_med = find_median(atk_forces)
    def_med = find_median(def_forces)
    
    for i in range(len(atk_forces)):
        atk_forces[i] = atk_forces[i] / atk_med * 2
        def_forces[i] /= def_med
            
    return atk_forces, def_forces

def model(club1, club2, sims = 10000):
    '''
    receives 2 clubs (forces) and returns the most likely result, probability of each result and probability for the game
    '''
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson.rvs(club1[0] - club2[1])
        goals2 = poisson.rvs(club2[0] - club1[1])
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [3]:
def championship(clubs, sims = 10000, year = 0):
    atk_forces, def_forces = find_forces(clubs, year = year)
    games = generate_games()
    results = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            club1 = (atk_forces[home], def_forces[home])
            club2 = (atk_forces[away], def_forces[away])
            _, result, _ = model(club1, club2, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results

In [4]:
clubs = generate_players(temps = 1)
atk_forces, def_forces = find_forces(clubs)

# exemplo
club1 = (atk_forces[ord('A') - 65], def_forces[ord('A') - 65])
club2 = (atk_forces[ord('B') - 65], def_forces[ord('B') - 65])
results, result, probs = model(club1, club2, sims = 10000)

print('Forças do clube 1:', club1)
print('Forças do clube 2:', club2)
print()
print('Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):')
print(results)
print()
print('O resultado mais provável é {} x {}.'.format(result[0], result[1]))
print('Além disso, a probabilidade do mandante é {}, enquanto a do visitante é de {} e a de empate é {}.'.format(probs[0], probs[2], probs[1]))

Forças do clube 1: (2.0489387940546506, 1.0038183456576013)
Forças do clube 2: (1.9951969883819045, 1.000071056859577)

Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):
[[1364 1293  656  200   60   15]
 [1258 1294  698  244   57   14]
 [ 703  732  360  119   33    5]
 [ 245  285  115   32    7    2]
 [  59   60   34   10    5    0]
 [  16   10   12    2    1    0]]

O resultado mais provável é 0 x 0.
Além disso, a probabilidade do mandante é 0.3542, enquanto a do visitante é de 0.3403 e a de empate é 0.3055.


In [5]:
df, results = championship(clubs, sims = 100)
df

,points,games,wins,draws,defeats,goals for,goals against,goal difference
S,66,38,17,15,6,26,15,11
H,62,38,14,20,4,27,17,10
O,58,38,13,19,6,23,16,7
D,51,38,12,15,11,21,20,1
G,51,38,10,21,7,22,19,3
F,51,38,8,27,3,22,17,5
A,50,38,10,20,8,21,19,2
N,50,38,9,23,6,18,15,3
T,49,38,11,16,11,21,22,-1
P,48,38,9,21,8,17,16,1


In [6]:
results

[['A', 1, 0, 'B'],
 ['A', 1, 0, 'C'],
 ['A', 0, 1, 'D'],
 ['A', 1, 1, 'E'],
 ['A', 1, 1, 'F'],
 ['A', 0, 0, 'G'],
 ['A', 0, 1, 'H'],
 ['A', 1, 1, 'I'],
 ['A', 1, 0, 'J'],
 ['A', 0, 1, 'K'],
 ['A', 1, 0, 'L'],
 ['A', 1, 0, 'M'],
 ['A', 0, 1, 'N'],
 ['A', 0, 1, 'O'],
 ['A', 0, 1, 'P'],
 ['A', 1, 1, 'Q'],
 ['A', 1, 1, 'R'],
 ['A', 1, 0, 'S'],
 ['A', 1, 1, 'T'],
 ['B', 0, 0, 'A'],
 ['B', 1, 0, 'C'],
 ['B', 1, 0, 'D'],
 ['B', 0, 0, 'E'],
 ['B', 1, 1, 'F'],
 ['B', 0, 0, 'G'],
 ['B', 0, 0, 'H'],
 ['B', 0, 0, 'I'],
 ['B', 0, 0, 'J'],
 ['B', 1, 1, 'K'],
 ['B', 0, 0, 'L'],
 ['B', 0, 0, 'M'],
 ['B', 1, 1, 'N'],
 ['B', 0, 0, 'O'],
 ['B', 0, 0, 'P'],
 ['B', 1, 1, 'Q'],
 ['B', 1, 0, 'R'],
 ['B', 0, 0, 'S'],
 ['B', 0, 1, 'T'],
 ['C', 0, 1, 'A'],
 ['C', 0, 0, 'B'],
 ['C', 1, 0, 'D'],
 ['C', 0, 0, 'E'],
 ['C', 1, 1, 'F'],
 ['C', 0, 1, 'G'],
 ['C', 0, 1, 'H'],
 ['C', 1, 0, 'I'],
 ['C', 1, 0, 'J'],
 ['C', 1, 0, 'K'],
 ['C', 1, 1, 'L'],
 ['C', 1, 0, 'M'],
 ['C', 1, 1, 'N'],
 ['C', 0, 1, 'O'],
 ['C', 0, 1,

Perceba que fazendo

$força~de~ataque~do~time = \sum força~dos~jogadores \text{ e } força~de~defesa~do~time = \sum \log{força~dos~jogadores},$

um time com maior força de ataque terá maior força de defesa, o que não é compatível com a realidade.

# Usando dois parâmetros por jogador

Cada jogador tem dois parâmetros: ataque e defesa

In [7]:
def generate_players(clubs = 20, players_per_club = 23):
    '''
    creates players
    '''
    n_players = clubs * players_per_club
    players_atk = truncnorm.rvs(0, 10, loc = 2, scale = 0.5, size = n_players)
    players_def = truncnorm.rvs(0, 10, loc = 1, scale = 0.5, size = n_players)
    players = {}
    for i in range(n_players):
        players['P{:04d}'.format(i)] = [round(players_atk[i], 8) if players_atk[i] < 100 else 100,
                                        round(players_def[i], 8) if players_def[i] < 100 else 100]
        
    return players

def change_clubs(clubs, changes):
    '''
    change clubs
    '''
    clubs_f = {}
    changing = []
    change = 0
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        np.random.shuffle(club_players)
        for j in range(changes[i]):
            changing.append(club_players[j])

    np.random.shuffle(changing)
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        ind = []
        for player in club_players:
            if player in changing:
                ind.append(club_players.index(player))

        for j in range(len(ind)):
            club_players[ind[j]] = changing[change]
            change += 1

        clubs_f[chr(i + 65)] = club_players

    return clubs_f

def generate_clubs(players, years = 15, n_clubs = 20, players_per_club = 23):
    '''
    receives players and allocates them to clubs
    '''
    clubs = {}
    all_players = list(players)
    for year in range(years):
        line_up = {chr(65 + club) : {} for club in range(n_clubs)}
        clubs[year] = line_up
        for club in range(n_clubs):
            if year == 0:
                clubs[year][chr(65 + club)] = all_players[players_per_club * club:players_per_club * (club + 1)]

            else:
                changes = poisson.rvs(players_per_club/4, size = n_clubs)
                for i in range(n_clubs):
                    if changes[i] > players_per_club:
                        changes[i] = players_per_club

                    clubs[year] = change_clubs(clubs[year - 1], changes)
    return clubs

In [8]:
years = 15
n_clubs = 20
players_per_club = 23
players = generate_players()
clubs = generate_clubs(players)

In [9]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

def model(club1, club2, players, sims = 10000):
    '''
    receives 2 clubs (line_up) and returns the most likely result, probability of each result and probability for the game
    '''
    atk1, def1 = find_forces(club1, players)
    atk2, def2 = find_forces(club2, players)
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson.rvs(atk1 / def2)
        goals2 = poisson.rvs(atk2 / def1)
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [10]:
def championship(clubs, players, sims = 10000, year = 0):
    games = generate_games()
    results = []
    line_up = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            
            club1 = clubs[year][chr(home + 65)].copy()
            np.random.shuffle(club1)
            home_line_up = club1[:11]
            
            club2 = clubs[year][chr(away + 65)].copy()
            np.random.shuffle(club2)
            away_line_up = club2[:11]
            
            _, result, _ = model(home_line_up, away_line_up, players, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            line_up.append([home_line_up, away_line_up])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results, line_up

In [11]:
table, results, line_up = championship(clubs, players, sims = 100, year = 0)

In [12]:
table

,points,games,wins,draws,defeats,goals for,goals against,goal difference
J,71,38,19,14,5,92,78,14
F,66,38,18,12,8,86,73,13
R,61,38,16,13,9,94,79,15
I,61,38,14,19,5,89,73,16
L,56,38,16,8,14,84,83,1
A,56,38,13,17,8,92,82,10
D,55,38,13,16,9,78,66,12
M,54,38,14,12,12,82,78,4
B,54,38,13,15,10,80,82,-2
H,53,38,15,8,15,87,89,-2


In [13]:
results

[['A', 1, 1, 'B'],
 ['A', 2, 2, 'C'],
 ['A', 2, 2, 'D'],
 ['A', 2, 3, 'E'],
 ['A', 3, 3, 'F'],
 ['A', 2, 2, 'G'],
 ['A', 2, 2, 'H'],
 ['A', 1, 1, 'I'],
 ['A', 3, 2, 'J'],
 ['A', 4, 1, 'K'],
 ['A', 3, 2, 'L'],
 ['A', 4, 2, 'M'],
 ['A', 2, 3, 'N'],
 ['A', 3, 3, 'O'],
 ['A', 4, 3, 'P'],
 ['A', 2, 4, 'Q'],
 ['A', 1, 2, 'R'],
 ['A', 5, 3, 'S'],
 ['A', 2, 5, 'T'],
 ['B', 2, 1, 'A'],
 ['B', 2, 2, 'C'],
 ['B', 1, 2, 'D'],
 ['B', 3, 2, 'E'],
 ['B', 1, 2, 'F'],
 ['B', 3, 2, 'G'],
 ['B', 1, 3, 'H'],
 ['B', 2, 2, 'I'],
 ['B', 2, 2, 'J'],
 ['B', 1, 1, 'K'],
 ['B', 4, 3, 'L'],
 ['B', 2, 2, 'M'],
 ['B', 2, 2, 'N'],
 ['B', 3, 3, 'O'],
 ['B', 2, 3, 'P'],
 ['B', 2, 2, 'Q'],
 ['B', 3, 2, 'R'],
 ['B', 5, 3, 'S'],
 ['B', 1, 5, 'T'],
 ['C', 2, 2, 'A'],
 ['C', 2, 2, 'B'],
 ['C', 1, 2, 'D'],
 ['C', 3, 2, 'E'],
 ['C', 2, 2, 'F'],
 ['C', 2, 2, 'G'],
 ['C', 5, 3, 'H'],
 ['C', 2, 2, 'I'],
 ['C', 3, 4, 'J'],
 ['C', 3, 1, 'K'],
 ['C', 2, 3, 'L'],
 ['C', 2, 3, 'M'],
 ['C', 1, 2, 'N'],
 ['C', 1, 1, 'O'],
 ['C', 2, 2,

In [14]:
line_up

[[['P0000',
   'P0013',
   'P0014',
   'P0009',
   'P0017',
   'P0003',
   'P0012',
   'P0015',
   'P0002',
   'P0019',
   'P0006'],
  ['P0024',
   'P0043',
   'P0045',
   'P0039',
   'P0036',
   'P0030',
   'P0026',
   'P0037',
   'P0025',
   'P0031',
   'P0033']],
 [['P0015',
   'P0022',
   'P0017',
   'P0016',
   'P0019',
   'P0005',
   'P0021',
   'P0003',
   'P0013',
   'P0014',
   'P0012'],
  ['P0048',
   'P0067',
   'P0051',
   'P0053',
   'P0055',
   'P0059',
   'P0062',
   'P0057',
   'P0046',
   'P0061',
   'P0060']],
 [['P0003',
   'P0017',
   'P0014',
   'P0012',
   'P0001',
   'P0004',
   'P0019',
   'P0000',
   'P0022',
   'P0020',
   'P0015'],
  ['P0082',
   'P0081',
   'P0085',
   'P0084',
   'P0072',
   'P0074',
   'P0086',
   'P0077',
   'P0089',
   'P0083',
   'P0087']],
 [['P0002',
   'P0004',
   'P0018',
   'P0015',
   'P0005',
   'P0010',
   'P0017',
   'P0009',
   'P0022',
   'P0016',
   'P0008'],
  ['P0098',
   'P0095',
   'P0096',
   'P0106',
   'P0111',
   'P0

# Engenharia reversa

In [15]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

# def find_players(players_er, codes):
#     players = {}
#     for code in codes:
#         if code not in players:
#             players[code] = players_er[codes.index(code)]
    
# #     for player in players_er:
# #         if player[0] not in players:
# #             players[player[0]] = players_er[players_er.index(player)][1:]
    
#     return players

def find_players(players_er, codes):
    players = {}
    for code in codes:
        if code not in players:
            players[code] = [players_er[2 * codes.index(code)], players_er[2 * codes.index(code) + 1]]
    
    return players

def likelihood(players_er, codes, results, line_up):
    '''
    players : parameter list
    results : list
    line_up : list
    '''
    players_er = find_players(players_er, codes)
    loglikelihood = 0
    for game in line_up:               
        ind = line_up.index(game)
        atk1, def1 = find_forces(game[0], players_er)
        atk2, def2 = find_forces(game[1], players_er)
        
        loglikelihood += poisson.logpmf(results[ind][1], atk1 / def2)
        loglikelihood += poisson.logpmf(results[ind][2], atk2 / def1)
        
    global k
    if k % 1000 == 0:
        print('Iteration: {}\nValue: {}\n'.format(k, -loglikelihood))
    k += 1
    
    return -loglikelihood

In [16]:
k = 0
players_er = {}
codes = []
for game in line_up:
    for player in game[0]:
        if player not in players_er:
            players_er[player] = [truncnorm.rvs(0, 10, loc = 3, scale = 1), truncnorm.rvs(0, 10, loc = 1, scale = 1)]
            codes.append(player)
    for player in game[1]:
        if player not in players_er:
            players_er[player] = [truncnorm.rvs(0, 10, loc = 1, scale = 1), truncnorm.rvs(0, 10, loc = 1, scale = 1)]
            codes.append(player)

# players_er = [players_er[player] for player in players_er]
aux = players_er.copy()
players_er = []
for i in aux:
    players_er.append(aux[i][0])
    players_er.append(aux[i][1])

lik = likelihood(players_er, codes, results, line_up)
lik

0


1239.9542920049994

In [17]:
result = minimize(likelihood, players_er, args = (codes, results, line_up), bounds = [(0, None) for player in players_er])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600


In [18]:
result

      fun: 1027.6276218229823
 hess_inv: <920x920 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.55978328e-02,  2.28283172e-02,  2.41243470e-02, -4.57021090e-03,
        4.59294825e-02,  8.46966941e-02, -3.07636585e-02, -1.14823707e-02,
        8.57198762e-03, -9.39053285e-03, -2.45790945e-02,  8.44465831e-02,
       -6.61657400e-03,  4.48153514e-02,  2.47382559e-02,  2.71256795e-02,
       -2.48519429e-02, -1.78033589e-02,  8.75388656e-03,  2.42835087e-02,
       -1.22781785e-02, -5.84350349e-03,  9.28139343e-02,  3.29237084e-02,
       -2.21461961e-02,  1.36856215e-01,  7.50333134e-04,  2.38969734e-02,
       -1.09594112e-02,  6.31871448e-02, -3.47881726e-03,  2.67618818e-02,
       -7.86712922e-03,  1.73486115e-02,  9.63154889e-02,  4.36102710e-02,
        7.30551619e-02,  5.75482776e-02, -1.95541362e-02,  4.13137971e-02,
        6.41193765e-03, -2.46018296e-02, -2.50111045e-02,  3.90173229e-02,
       -2.48064679e-02,  3.89036359e-02, -1.55523595e-02, -1.91903383e-02,

In [19]:
result.nit

15

In [20]:
result.x

array([8.26340675e+00, 3.08398690e+00, 2.53124680e-01, 2.64024440e+00,
       4.69658022e-01, 0.00000000e+00, 6.39466196e+00, 3.87716420e+00,
       1.04046176e+00, 4.54272725e+00, 3.94222102e+00, 0.00000000e+00,
       4.07494104e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.23852279e+00, 1.22064819e+00, 3.75773235e+00, 1.79152835e+00,
       3.61021991e+00, 5.96582260e+00, 0.00000000e+00, 3.52887336e+00,
       5.25417863e+00, 0.00000000e+00, 2.73690660e+00, 3.21084358e+00,
       0.00000000e+00, 1.83053780e+00, 4.06675093e+00, 3.71524919e+00,
       2.31861120e+00, 7.57651117e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.70767710e+00, 1.96849767e+00, 4.72685283e+00,
       0.00000000e+00, 8.71005348e+00, 2.67628553e+00, 2.71862881e+00,
       7.40354593e-01, 0.00000000e+00, 5.43001711e+00, 1.75997652e+00,
       2.89734770e+00, 4.66876093e+00, 1.07273037e+00, 5.18248805e+00,
       7.35816431e+00, 5.12912839e+00, 1.48454070e+00, 4.79584971e-01,
      

In [21]:
find_players(result.x, codes)

{'P0000': [8.2634067532913, 3.0839869007603276],
 'P0013': [0.25312468048743886, 2.640244400954797],
 'P0014': [0.46965802223123776, 0.0],
 'P0009': [6.394661963069013, 3.8771642003292026],
 'P0017': [1.0404617594430003, 4.542727249319141],
 'P0003': [3.942221023525008, 0.0],
 'P0012': [4.074941037797824, 0.0],
 'P0015': [0.0, 0.0],
 'P0002': [1.238522793947448, 1.2206481908207554],
 'P0019': [3.7577323493979864, 1.7915283455310185],
 'P0006': [3.610219912668873, 5.965822604885921],
 'P0024': [0.0, 3.5288733619982593],
 'P0043': [5.254178631857496, 0.0],
 'P0045': [2.7369066025176667, 3.2108435799534023],
 'P0039': [0.0, 1.8305378006201471],
 'P0036': [4.0667509309558385, 3.7152491902854847],
 'P0030': [2.318611195115624, 7.576511174450727],
 'P0026': [0.0, 0.0],
 'P0037': [0.0, 1.7076770981473766],
 'P0025': [1.968497670327826, 4.726852825031364],
 'P0031': [0.0, 8.710053478468652],
 'P0033': [2.676285529736747, 2.7186288051659395],
 'P0022': [0.7403545932136092, 0.0],
 'P0016': [5.43